In [1]:
import ee
import geemap
import time
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

#geemap.ee_search()
ee.Initialize()
# Create a GIS object
gis = GIS('PRO')



# %%
Map = geemap.Map()

In [2]:
Map.layers

(TileLayer(attribution='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors', base=True, max_zoom=19, min_zoom=1, name='OpenStreetMap.Mapnik', options=['attribution', 'bounds', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms', 'zoom_offset'], url='https://tile.openstreetmap.org/{z}/{x}/{y}.png'),
 TileLayer(attribution='Google', max_zoom=22, name='Google Maps', options=['attribution', 'bounds', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms', 'zoom_offset'], url='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}'))

In [3]:
ee.Authenticate()

True

In [4]:
import ee
import geemap
from datetime import datetime

# Initialize Earth Engine
ee.Initialize()


# Define the bound area
sppp = ee.FeatureCollection('users/raulperezastorga/Limites_SPPP')

# Get the current date
t2 = datetime.now().strftime('%Y-%m-%d')

# Calculate the dates
ti = '2023-08-01'
t1 = ee.Date(t2).advance(-4, 'week')
t0 = ee.Date(t2).advance(-1, 'day')

# Load the FIRMS dataset
dataset = ee.ImageCollection('FIRMS')

# Filter the FIRMS dataset for the past 1 week
fires1W = dataset.select('T21') \
                  .filterDate(ti, t2) \
                  .filterBounds(sppp)

# Filter the FIRMS dataset for the past 6 hours
fires6H = dataset.select('T21') \
                  .filterDate(t0, t2) \
                  .filterBounds(sppp)

# Visualization parameters
firesVis = {
    'min': 70,
    'max': 250,
    'palette': ['#ff0000', '#ff5a00', '#ff9a00', '#ffce00', '#ffe808', '#ffffff', '#a41fd6']
}

fireVisReclass = {
    'min': 1,
    'max': 3,
    'palette': ['#ff0000', '#ff9a00', '#ffe808', '#a41fd6']
}

# Calculate the maximum temperature for the past 1 week and convert to °C
firemx1W = fires1W.max().clip(sppp).subtract(273.15)


# Reclassify the maximum temperature for the past 1 week
firemx1Wr = firemx1W.gte(80).add(firemx1W.gte(100)).add(firemx1W.gte(150))


# Create a map centered on the bound area
Map = geemap.Map(center=(sppp.geometry().centroid().getInfo()['coordinates'][::-1]))

# Add the reclassified fires layer to the map
Map.addLayer(firemx1Wr, fireVisReclass, 'Fires')

# Display the map
Map

OSError: CURRENT